churn results when a customer cancels their subscription or relationship with a company. preventing cutomer churn is critical, and some estimates suggest that acquiring a new customer can be as much as 25 times as costly as keeping an existing one.
    The first steps to reducing churn is understanding the extent to which your customers are churning. this templates will cover three churn metrics you can use to know how many users and how much revenue you are losing to churn. it is appropriate for subscription data or other forms of recurring revenue data.
    Imports and data preprocessing
    The following cell imports the package necessary to import and manipulate user payment data. it also loads and prepares the example data.
    The example data in this templates contains the payment date, user id, and price paid for a subscription services.

In [4]:
#import package
import numpy as np
import pandas as pd
import plotly.express as px

#set the path to your file
path = "payments.csv"

#set your date column
data_col = "payment_date"

#import churn data
try:
    payments = pd.read_csv(path, parse_dates=[data_col])
except FileNotFoundError:
    print(f"File not found: {path}")
    # Handle the error or provide an alternative path
    # For example, you can set payments to None or load a default dataset
    payments = None

if payments is not None:
    #categorize payment periods(optional)
    payments["payment_period"] = pd.to_datetime(payments[data_col]) + pd.offsets.MonthBegin(-1)

    #preview the data
    display(payments)

,payment_date,userid,price,payment_period
0,2016-08-10 00:00:00+00:00,0014163a53056db3f10e,5000,2016-08-01 00:00:00+00:00
1,2016-09-10 00:00:00+00:00,0014163a53056db3f10e,5000,2016-09-01 00:00:00+00:00
2,2016-10-10 00:00:00+00:00,0014163a53056db3f10e,5000,2016-10-01 00:00:00+00:00
3,2016-05-18 00:00:00+00:00,00164e47967b306239e8,5000,2016-05-01 00:00:00+00:00
4,2016-06-07 00:00:00+00:00,0016e5b445a29e9cbfcc,5000,2016-06-01 00:00:00+00:00
...,...,...,...,...
8769,2016-07-27 00:00:00+00:00,ffdadb16ec552ff35d2a,5000,2016-07-01 00:00:00+00:00
8770,2016-08-27 00:00:00+00:00,ffdadb16ec552ff35d2a,5000,2016-08-01 00:00:00+00:00
8771,2016-09-27 00:00:00+00:00,ffdadb16ec552ff35d2a,5000,2016-09-01 00:00:00+00:00
8772,2016-07-20 00:00:00+00:00,ffdc1240,5000,2016-07-01 00:00:00+00:00


Net retention rate
Calculating the net retention rate
The first churn metric that we will calculate is the net retention rate.
The net retention rate is the percentage of recurring revenue that comes from the customers who were subscribed at the start of a period.
       Net retention = MRRretained account /
                        MRRstart
 Note that this calculation does not include any revenue from new subscriber. it does include the effect of churns, upsells(e.g, customers upgrading their subscription) and downsells(e.g, customers reducing their subscription tier), because it includes upsells, the net retention rate is a less specific measure of churn than other measures. it is, however a good overall metric that captures churn, upsells and downsells that can be easily interpreted by stakeholders(such as investors)
        In this and the following cells that defines the metrics functions, you will need to update the Datasets() if you wish to use a period other than monthly.


In [5]:
import pandas as pd

# Sample data for demonstration 
data = {
    "userid": [1, 2, 1, 2, 3, 4],
    "payment_period": pd.to_datetime(["2023-01-01", "2023-01-01", "2023-02-01", "2023-02-01", "2023-02-01", "2023-02-01"]),
    "price": [100, 200, 150, 250, 300, 400]
}
payment = pd.DataFrame(data)

# Define a function to calculate the next retention rate
def nrr(x, dataframe):
    # Identify current and previous period based on group.
    current_period = x["payment_period"].max()
    previous_period = current_period - pd.DateOffset(months=1)
    
    # Calculate the previous period's recurring revenue
    previous_period_rr = dataframe[dataframe["payment_period"] == previous_period]["price"].sum()
    
    # Identify the users in the previous period
    previous_period_users = dataframe[dataframe["payment_period"] == previous_period]["userid"].unique()
    
    # Calculate the current period's recurring revenue that comes from the previous period's users
    current_period_retained_rr = dataframe[(dataframe["userid"].isin(previous_period_users)) & (dataframe["payment_period"] == current_period)]["price"].sum()
    
    # Calculate and return the net retention rate
    try:
        nrr = current_period_retained_rr / previous_period_rr
        return nrr
    except ZeroDivisionError:
        return None

# Group the data by payment period and calculate the net retention rate
nrr_data = (
    payment.groupby("payment_period")
    .apply(nrr, dataframe=payment)
    .reset_index(name="nrr")
)

# Preview the data
nrr_data

,payment_period,nrr
0,2023-01-01,NaN
1,2023-02-01,1.333333


Note you may notice low retention and high churn rates in the final months of data(february 2017). This is because the data is incompleted for this final period.

Plotting the net retention rate
        We can plot the net retention rate using a line chart from plotly which provides an interactive visualization of each  period.

In [6]:
import plotly.express as px
import pandas as pd

# Sample data for demonstration purpose
nrr_data = pd.DataFrame({
    "payment_period": ["Q1", "Q2", "Q3", "Q4"],
    "nrr": [100, 110, 105, 115]
})

fig = px.line(
    nrr_data,
    x="payment_period",
    y="nrr",
    title="Net Retention Rate by Period"
)
fig.show()

Standard account churn
Calculating standard account churn
The next churn metric we will calculate is "standard account churn". Standard account churn is the proportion of customers who cancel to the total number of customers at the start of a period. A customer is only considered a churn if they cancel all subscriptions, and therefore a down sell or reduction in the number of subscriptions a customer holds does not constitute churn.
             Standard Churn Rate = Numberchurns /
                                   Number start
 
This metric does not consider upsells and down sells and provides an easy-to-interpret measure of churn. Standard account churn is a good metric to use when you have a free service or when all customers pay the same price.

In [31]:
import pandas as pd
import numpy as np

# Sample data for demonstration purposes
data = {
    "userid": [1, 2, 3, 4, 1, 2, 3, 5],
    "payment_period": pd.to_datetime([
        "2023-01-01", "2023-01-01", "2023-01-01", "2023-01-01",
        "2023-02-01", "2023-02-01", "2023-02-01", "2023-02-01"
    ])
}

# Create a DataFrame
payment = pd.DataFrame(data)

# Define a function to calculate the standard account churn
def std_churn(x, dataframe):
    # Identify current and previous period based on group
    current_period = x.name
    previous_period = current_period - pd.DateOffset(months=1)
    
    # Calculate the previous period's total number of users
    previous_period_user_count = dataframe[
        dataframe["payment_period"] == previous_period]["userid"].nunique()
    
    # Identify the users in the previous and current periods
    previous_period_users = dataframe[dataframe["payment_period"] == previous_period]["userid"].unique()
    current_period_users = dataframe[dataframe["payment_period"] == current_period]["userid"].unique()
    
    # Calculate the number of churned users
    churns = len(np.setdiff1d(previous_period_users, current_period_users))
    
    # Calculate and return the standard churn
    try:
        std_churn = churns / previous_period_user_count
        return std_churn
    except ZeroDivisionError:
        return None

# Group the data by payment period and calculate the standard churn rate
std_column_data = (
    payment.groupby("payment_period")
    .apply(std_churn, dataframe=payment)
    .reset_index(name="std_churn")
)

# Review the data
std_column_data

,payment_period,std_churn
0,2023-01-01,NaN
1,2023-02-01,0.25


plotting the standard account churn
we can give again plot the churn by using a plotly line chart.

In [2]:
import plotly.express as px
import pandas as pd

#Sample data to define std_churn_data
data = {
    "payment_period": ["2021-01", "2021-02", "2021-03", "2021-04"], 
    "std_churn": [0.1, 0.15, 0.13, 0.2]
}
std_churn_data = pd.DataFrame(data)

fig = px.line(
      std_churn_data,
      x = "payment_period", 
      y = "std_churn",
      title = "Standard Account Churn by Period",
)

fig.show()

Monthly recurring revenue churn
Calculating monthly recurring revenue churn
The final churn metric to cover is "monthly recurring revenue", a variant of net retention rate. Whereas net retention rate includes upsells, monthly recurring revenue does not, as technically, these do not constitute churn.
            MRR churn = MRRchurned account+MRRdownsell /
                                MRRstart

 
Monthly recurring revenue churn is a useful metric if you have a variety of different prices (i.e., different pricing tiers). It is considered a more accurate definition of churn because it focuses exclusively on down sells and total churn.

In [1]:
import pandas as pd
import numpy as np

# Sample data for demonstration purposes
data = {
    "userid": [1, 2, 1, 3, 2, 4],
    "payment_period": pd.to_datetime(["2023-01-01", "2023-01-01", "2023-02-01", "2023-02-01", "2023-02-01", "2023-02-01"]),
    "price": [100, 200, 150, 300, 250, 400]
}
payments = pd.DataFrame(data)

# Define a function to calculate the periodly recurring revenue churn
def mrr(x, dataframe):
    # Identify current and previous period based on group
    current_period = x["payment_period"].max()
    previous_period = current_period - pd.DateOffset(months=1)
    # Identify total recurring revenue from previous period
    previous_period_rr = dataframe[dataframe["payment_period"] == previous_period]["price"].sum()
    # Identify the users in the previous and current periods
    previous_period_users = dataframe[dataframe["payment_period"] == previous_period]["userid"].unique()
    current_period_users = dataframe[dataframe["payment_period"] == current_period]["userid"].unique()
    # Identify retained and churned users
    retained_users = np.intersect1d(previous_period_users, current_period_users)
    churned_users = np.setdiff1d(previous_period_users, current_period_users)
    # Identify revenue of retained users in each period
    current_period_retained_rr = dataframe[
        (dataframe["userid"].isin(retained_users))
        & (dataframe["payment_period"] == current_period)
    ]["price"].values
    previous_period_retained_rr = dataframe[
        (dataframe["userid"].isin(retained_users))
        & (dataframe["payment_period"] == previous_period)
    ]["price"].values
    # Define mask to filter arrays for downsells
    downsells = previous_period_retained_rr > current_period_retained_rr
    # Calculate total downsell loss
    downsell_loss = np.sum(
        previous_period_retained_rr[downsells] - current_period_retained_rr[downsells]
    )
    # Calculate loss to users who churned
    churn_loss = dataframe[
        (dataframe["userid"].isin(churned_users))
        & (dataframe["payment_period"] == previous_period)
    ]["price"].sum()
    # Caculate and return the monthly recurring revenue
    try:
        mrr = (downsell_loss + churn_loss) / previous_period_rr
        return mrr
    except:
        return None

# Group the data by payment period and calculate the monthly recurring revenue churn
mrr_data = (
    payments.groupby("payment_period")
    .apply(mrr, dataframe=payments)
    .reset_index(name="mrr")
)

# Review the data
mrr_data

,payment_period,mrr
0,2023-01-01,NaN
1,2023-02-01,0.0


Plotting monthly recurring revenue churn
As with the previous metrics, we can use a Plotly line chart to plot the monthly recurring revenue churn.

In [2]:
import plotly.express as px
import pandas as pd

# Sample data for demonstration purposes
mrr_data = pd.DataFrame({
    "payment_period": ["2021-01", "2021-02", "2021-03", "2021-04"],
    "mrr": [1000, 1100, 1050, 1150]
})

fig = px.line(
    mrr_data,
    x="payment_period",
    y="mrr",
    title="Recurring Revenue Churn by Period"
)

fig.show()